**Building Tables for the DataBase**

In [34]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [35]:
%%sql
sqlite:///CourseData.db
    
DROP TABLE IF EXISTS COURSE_OFFERING;
DROP TABLE IF EXISTS CATALOG_DESC;
DROP TABLE IF EXISTS INSTRUCTOR;
DROP TABLE IF EXISTS COURSE_MEETING;
DROP TABLE IF EXISTS CATALOG_YEAR_FILL;

Done.
Done.
Done.
Done.
Done.


[]

In [36]:
%%sql
CREATE TABLE CATALOG_DESC (
    catalog_desc_id INTEGER PRIMARY KEY,
    catalog_id TEXT NOT NULL,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL,
    course_title TEXT NOT NULL,
    credits TEXT NOT NULL,
    prereqs TEXT,
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,
    description TEXT,
    catalog_year TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.


[]

In [37]:
%%sql
CREATE TABLE INSTRUCTOR (
    instructor_id INTEGER PRIMARY KEY,
    primary_instructor TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.


[]

In [38]:
%%sql
CREATE TABLE COURSE_OFFERING (
    course_offering_id INTEGER PRIMARY KEY,
    CRN TEXT NOT NULL,
    catalog_desc_id INTEGER,
    term TEXT NOT NULL,
    section TEXT NOT NULL,
    title TEXT NOT NULL,
    credits INTEGER NOT NULL,
    cap INTEGER NOT NULL,
    act INTEGER NOT NULL,
    rem INTEGER NOT NULL, 
    meetings TEXT NOT NULL, 
    timecodes TEXT NOT NULL,
    catalog_year TEXT,
    instructor_id INTEGER NOT NULL,
    FOREIGN KEY (instructor_id) REFERENCES INSTRUCTOR (instructor_id),
    FOREIGN KEY (catalog_desc_id) REFERENCES CATALOG_DESC (catalog_desc_id)
);

 * sqlite:///CourseData.db
Done.


[]

In [39]:
%%sql
CREATE TABLE COURSE_MEETING (
    meet_id INTEGER PRIMARY KEY,
    course_offering_id INTEGER NOT NULL,
    CRN TEXT NOT NULL,
    term TEXT NOT NULL,
    location TEXT, 
    day TEXT NOT NULL, 
    start TEXT NOT NULL,
    end TEXT NOT NULL,
    FOREIGN KEY (course_offering_ID) REFERENCES COURSE_OFFERING (course_offering_ID)
);

 * sqlite:///CourseData.db
Done.


[]

In [40]:
%%sql
CREATE TABLE CATALOG_YEAR_FILL (
    term TEXT NOT NULL,
    catalog_year TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.


[]

In [41]:
%%sql
INSERT INTO CATALOG_YEAR_FILL (term, catalog_year) VALUES
('Fall2017','2017_2018'), ('Winter2018','2017_2018'), ('Spring2018','2017_2018'), ('SpringBreak2018','2017_2018')
 , ('Summer2018','2017_2018'), ('Fall2018','2018_2019'), ('Winter2019','2018_2019'), ('Spring2019','2018_2019');

 * sqlite:///CourseData.db
8 rows affected.


[]

In [42]:
%%sql
INSERT INTO INSTRUCTOR (primary_instructor)
SELECT DISTINCT primary_instructor
FROM import_courses;

 * sqlite:///CourseData.db
1105 rows affected.


[]

In [43]:
%%sql
INSERT INTO CATALOG_DESC (catalog_id, program_code, program_name, course_title, credits, prereqs, coreqs, fees, attributes, description, catalog_year)
SELECT catalog_id, program_code, program_name, course_title, credits, prereqs, coreqs, fees, attributes, description, catalog_year
FROM import_catalogs;

 * sqlite:///CourseData.db
4441 rows affected.


[]

In [45]:
%%sql
INSERT INTO COURSE_OFFERING (CRN, term, section, title, credits, cap, act, rem, meetings, timecodes, instructor_id, catalog_desc_id, catalog_year)
SELECT CRN, term, section, title, import_courses.credits, cap, act, rem, meetings, timecodes, instructor_id, catalog_desc_id, CATALOG_YEAR_FILL.catalog_year
    FROM import_courses
    JOIN INSTRUCTOR ON (import_courses.primary_instructor=INSTRUCTOR.primary_instructor)
    LEFT JOIN CATALOG_YEAR_FILL USING (term)
    LEFT JOIN CATALOG_DESC ON (import_courses.catalog_id=CATALOG_DESC.catalog_id AND CATALOG_YEAR_FILL.catalog_year=CATALOG_DESC.catalog_year);

 * sqlite:///CourseData.db
15955 rows affected.


[]

In [46]:
%%sql
INSERT INTO COURSE_MEETING (course_offering_id, CRN, term, location, day, start, end)
SELECT DISTINCT course_offering_id, import_course_meetings.CRN, import_course_meetings.term, location, day, start, end
    FROM import_course_meetings
    JOIN COURSE_OFFERING ON (import_course_meetings.CRN=COURSE_OFFERING.CRN AND import_course_meetings.term=COURSE_OFFERING.term);

 * sqlite:///CourseData.db
311159 rows affected.


[]

In [47]:
%%sql
SELECT COUNT(*)
FROM import_course_meetings;

 * sqlite:///CourseData.db
Done.


COUNT(*)
317339


In [56]:
%%sql
SELECT DISTINCT (primary_instructor) AS Teacher, COUNT(section) AS No_Sections
FROM COURSE_OFFERING
JOIN INSTRUCTOR ON (COURSE_OFFERING.instructor_id=INSTRUCTOR.instructor_id)
WHERE act > 10 AND term = 'Fall2018'
GROUP BY primary_instructor
ORDER BY No_sections DESC
LIMIT 10;

 * sqlite:///CourseData.db
Done.


Teacher,No_Sections
Nicole Elizabeth Heller,53
Anthony J. DeCristofaro,5
Christine Earls,5
Christine R. Rodriguez,5
Michael Walden Creane,5
Nancy A. Moriber,5
Qin Zhang,5
Sonya R. Huber,5
William F. Vasquez Mazariegos,5
Anderson Weekes,4


In [66]:
%%sql
DROP TABLE import_courses;
DROP TABLE import_course_meetings;
DROP TABLE import_catalogs;

 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [69]:
%%sql
SELECT primary_instructor AS Teacher, title AS Class, day, start, end
FROM COURSE_OFFERING
JOIN INSTRUCTOR ON (COURSE_OFFERING.instructor_id=INSTRUCTOR.instructor_id)
JOIN COURSE_MEETING ON (COURSE_OFFERING.course_offering_id=COURSE_MEETING.course_offering_id)
WHERE COURSE_OFFERING.CRN = '39006' AND COURSE_OFFERING.term = 'Spring2019';

 * sqlite:///CourseData.db
Done.


Teacher,Class,day,start,end
Christopher L. Huntley,Databases for Business Analytics,T,2019-01-29T18:30:00,2019-01-29T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-05T18:30:00,2019-02-05T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-12T18:30:00,2019-02-12T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-26T18:30:00,2019-02-26T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-03-05T18:30:00,2019-03-05T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-03-12T18:30:00,2019-03-12T21:30:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-01-29T09:00:00,2019-01-29T16:00:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-05T09:00:00,2019-02-05T16:00:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-12T09:00:00,2019-02-12T16:00:00
Christopher L. Huntley,Databases for Business Analytics,T,2019-02-26T09:00:00,2019-02-26T16:00:00


In [70]:
%%sql
SELECT DISTINCT (primary_instructor) AS Teacher, COUNT(section) AS No_Sections
FROM COURSE_OFFERING
JOIN INSTRUCTOR ON (COURSE_OFFERING.instructor_id=INSTRUCTOR.instructor_id)
WHERE act > 10 AND term = 'Fall2018'
GROUP BY primary_instructor
ORDER BY No_sections DESC
LIMIT 10;

 * sqlite:///CourseData.db
Done.


Teacher,No_Sections
Nicole Elizabeth Heller,53
Anthony J. DeCristofaro,5
Christine Earls,5
Christine R. Rodriguez,5
Michael Walden Creane,5
Nancy A. Moriber,5
Qin Zhang,5
Sonya R. Huber,5
William F. Vasquez Mazariegos,5
Anderson Weekes,4


In [105]:
#%%sql
SELECT DISTINCT(COURSE_OFFERING.instructor_id), primary_instructor AS Teacher, program_name AS Department
FROM COURSE_OFFERING
JOIN INSTRUCTOR ON (INSTRUCTOR.instructor_id=COURSE_OFFERING.instructor_id) 
JOIN CATALOG_DESC ON (CATALOG_DESC.catalog_desc_id=COURSE_OFFERING.catalog_desc_id)
ORDER BY COURSE_OFFERING.instructor_id


SyntaxError: invalid syntax (<ipython-input-105-68ec2f406eab>, line 2)

In [106]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///CourseData.db")
with engine.connect() as conn, conn.begin():
    teacher_department = pd.read_sql('SELECT DISTINCT(COURSE_OFFERING.instructor_id), primary_instructor AS Teacher, program_name AS Department FROM COURSE_OFFERING JOIN INSTRUCTOR ON (INSTRUCTOR.instructor_id=COURSE_OFFERING.instructor_id) JOIN CATALOG_DESC ON (CATALOG_DESC.catalog_desc_id=COURSE_OFFERING.catalog_desc_id) ORDER BY COURSE_OFFERING.instructor_id;', conn) 

In [107]:
teacher_department

,instructor_id,Teacher,Department
0,1,Michael P. Coyne,Accounting
1,2,Rebecca I. Bloch,Accounting
2,3,Paul Caster,Accounting
3,4,Jo Ann Drusbosky,Accounting
4,5,Arleen N. Kardos,Accounting
5,6,Scott M Brenner,Accounting
6,8,Bruce Bradford,Accounting
7,9,Milo W. Peck,Accounting
8,10,Stephen E. Yost,Accounting
9,10,Stephen E. Yost,Management


In [108]:
teacher_department.to_csv('teachers.csv', index=False)